In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install playwright
!playwright install-deps
!playwright install chromium

Installing dependencies...
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-freefont-ttf is already the newest version (20120503-10build1

In [14]:
# 비동기 실행을 위한 nest_asyncio 설정
import nest_asyncio
nest_asyncio.apply()

# 1월28일 뉴스(30개)

In [19]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
from datetime import datetime
import os
import re
import nest_asyncio

# Colab 환경 설정
nest_asyncio.apply()

# 경로 설정
SAVE_PATH = "/content/drive/MyDrive/DILAB/OK_Data/"
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

def clean_news_content(text):
    """
    RAG 성능 향상을 위해 뉴스 본문에서 노이즈를 제거합니다.
    """
    if not text: return ""

    # 1. 기자 정보 및 대괄호 문구 제거 (예: [스타뉴스 | 최혜진 기자])
    text = re.sub(r'\[.*?기자\]', '', text)
    text = re.sub(r'\[.*?\|.*?\]', '', text)
    text = re.sub(r'\[.*?=.*?\]', '', text)

    # 2. 이미지 관련 불필요한 문구 제거
    text = re.sub(r'원본 이미지 보기', '', text)
    text = re.sub(r'[^\s]*?\/사진=.*?\s', ' ', text)
    text = re.sub(r'사진=.*?\s', ' ', text)
    text = re.sub(r'ⓒ.*?\s', ' ', text)

    # 3. 이메일 주소 및 저작권 관련 문구 제거
    text = re.sub(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', '', text)
    text = re.sub(r'Copyright.*?Reserved\.', '', text, flags=re.IGNORECASE)
    text = re.sub(r'무단.*?금지\.?', '', text)
    text = re.sub(r'All rights reserved\.', '', text, flags=re.IGNORECASE)

    # 4. 특수문자 정제 및 연속 공백 통합
    text = re.sub(r'\s+', ' ', text).strip()

    return text

async def crawl_all_naver_news(test_mode=False): # 기본값을 False로 변경하여 전체 수집
    async with async_playwright() as p:
        # 브라우저 실행
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (iPhone; CPU iPhone OS 14_7_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Mobile/15E148 Safari/604.1"
        )
        page = await context.new_page()

        print("🔍 랭킹 페이지 접속 중...")
        await page.goto("https://m.entertain.naver.com/ranking", wait_until="domcontentloaded")

        # 모든 뉴스가 로딩되도록 아래로 스크롤
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(2000)

        # 1. 뉴스 링크 수집
        links = await page.eval_on_selector_all(
            "a:has([class*='NewsItem_title']), a[class*='mfc_modmarginfixed_margin_thumb_link']",
            "elements => elements.map(el => el.href)"
        )
        unique_links = list(dict.fromkeys([link for link in links if 'article' in link]))

        if test_mode:
            print(f"🧪 테스트 모드입니다. 상위 3개만 수집합니다.")
            unique_links = unique_links[:3]
        else:
            print(f"🚀 전체 수집 모드 시작! 총 {len(unique_links)}개의 기사를 수집합니다.")

        news_results = []

        # 2. 개별 기사 순회 수집
        for i, link in enumerate(unique_links):
            try:
                # 기사 페이지 접속
                await page.goto(link, wait_until="domcontentloaded", timeout=40000)
                await page.wait_for_timeout(1500) # 서버 부하 방지 및 렌더링 대기

                # 제목, 날짜, 본문 추출 (.first 사용으로 엄격 모드 방지)
                title = await page.locator("h2[class*='ArticleHead_article_title']").first.inner_text()
                date = await page.locator("em.date").first.inner_text()

                # 본문 추출 시도
                content_loc = page.locator("div._article_content, #comp_news_article")
                raw_content = await content_loc.first.inner_text()

                # 정제 함수 적용
                cleaned_content = clean_news_content(raw_content)

                if len(cleaned_content) > 20: # 내용이 너무 적은 기사는 제외
                    news_results.append({
                        "title": title,
                        "date": date,
                        "content": cleaned_content,
                        "url": link
                    })
                    print(f"✅ [{i+1}/{len(unique_links)}] 완료: {title[:20]}...")
                else:
                    print(f"⚠️ [{i+1}/{len(unique_links)}] 건너뜀 (내용 부족)")

            except Exception as e:
                print(f"❌ [{i+1}/{len(unique_links)}] 실패: {link} (사유: {str(e)[:50]})")

        # 3. 데이터 저장
        if news_results:
            df = pd.DataFrame(news_results)
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            file_name = f"full_cleaned_news_{timestamp}.csv"
            save_full_path = os.path.join(SAVE_PATH, file_name)

            df.to_csv(save_full_path, index=False, encoding='utf-8-sig')

            print("\n" + "="*50)
            print(f"🎉 크롤링 완료!")
            print(f"📁 총 수집 기사: {len(news_results)}개")
            print(f"💾 저장 경로: {save_full_path}")
            print("="*50)
        else:
            print("❌ 수집된 데이터가 없습니다.")

        await browser.close()

# 전체 모드로 실행
asyncio.run(crawl_all_naver_news(test_mode=False))

🔍 랭킹 페이지 접속 중...
🚀 전체 수집 모드 시작! 총 30개의 기사를 수집합니다.
✅ [1/30] 완료: '이혼' 박지윤·최동석, 상간 맞소송...
✅ [2/30] 완료: "상대적 박탈감"..'탈세' 차은우,...
✅ [3/30] 완료: 민희진 측 "어도어, 다니엘만 전속계...
✅ [4/30] 완료: 노유정, 식당서 설거지까지…이혼·피싱...
✅ [5/30] 완료: 유호정 두문불출했던 이유 “몇년간 칩...
✅ [6/30] 완료: [단독] 차은우 992字 입장 발표,...
✅ [7/30] 완료: 소송도 이겼는데..어쿠스틱콜라보 모수...
✅ [8/30] 완료: 결국 '민희진 vs 뉴진스 멤버' 됐...
✅ [9/30] 완료: [단독] '은애하는 도적님아' 후속작...
✅ [10/30] 완료: 민희진 측 "어도어 복귀 원했다"…"...
✅ [11/30] 완료: 차은우, 군대서도 쫓겨나나..“군악대...
✅ [12/30] 완료: 또 옥주현? ‘캐스팅 독식’ 논란에 ...
✅ [13/30] 완료: 서우림, 극단적 선택으로 떠난 子…전...
✅ [14/30] 완료: 28기 정숙♥상철, '임신 6개월차'...
✅ [15/30] 완료: 아빠 윤민수는 '이혼', 아들 윤후는...
✅ [16/30] 완료: "하루 매출 1억3천"…'두쫀쿠' 최...
✅ [17/30] 완료: 종영 딱 1회 남았는데 뒤늦게 아쉬움...
✅ [18/30] 완료: '김용건 며느리' 황보라, 남편만 모...
✅ [19/30] 완료: ‘러브캐처’ 김지연, 아이 발 사진 ...
✅ [20/30] 완료: 이수지, 소식가 인증 "요즘 살쪄서 ...
✅ [21/30] 완료: 김승수, 박세리와 결혼설에 입 열었다...
✅ [22/30] 완료: 하준수♥안가연, 외도 논란 이겨내고 ...
✅ [23/30] 완료: "이분을 뵙게 될 줄이야" 녹화장 술...
✅ [24/30] 완료: '자택서 숨진 채 발견' 유명 유튜버...
✅ [25/30] 완료: "누가봐도 이동건 딸" 조윤희 딸 로...
✅ [26/30]

# 1월 22일부터 28일치 크롤링

In [20]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
from datetime import datetime, timedelta
import os
import re
import nest_asyncio

# Colab 환경 설정
nest_asyncio.apply()

# 경로 설정
SAVE_PATH = "/content/drive/MyDrive/DILAB/OK_Data/"
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

def clean_news_content(text):
    """뉴스 본문 노이즈 제거 함수"""
    if not text: return ""
    text = re.sub(r'\[.*?기자\]|\[.*?\|.*?\]|\[.*?=.*?\]', '', text)
    text = re.sub(r'원본 이미지 보기|[^\s]*?\/사진=.*?\s|사진=.*?\s|ⓒ.*?\s', ' ', text)
    text = re.sub(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', '', text)
    text = re.sub(r'Copyright.*?Reserved\.|무단.*?금지\.?|All rights reserved\.', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

async def crawl_7days_news():
    # 1. 수집할 날짜 리스트 생성 (2026-01-22 ~ 2026-01-28)
    start_date = datetime(2026, 1, 22)
    date_list = [(start_date + timedelta(days=x)).strftime("%Y%m%d") for x in range(7)]

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (iPhone; CPU iPhone OS 14_7_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Mobile/15E148 Safari/604.1"
        )
        page = await context.new_page()

        all_news_data = []

        # 2. 날짜별 루프 시작
        for target_date in date_list:
            print(f"\n📅 [{target_date}] 날짜 기사 수집 시작...")
            ranking_url = f"https://m.entertain.naver.com/ranking?rankingDate={target_date}"

            try:
                await page.goto(ranking_url, wait_until="domcontentloaded")
                await page.mouse.wheel(0, 5000) # 하단 기사까지 로딩
                await page.wait_for_timeout(2000)

                # 해당 날짜의 뉴스 링크들 수집
                links = await page.eval_on_selector_all(
                    "a:has([class*='NewsItem_title']), a[class*='mfc_modmarginfixed_margin_thumb_link']",
                    "elements => elements.map(el => el.href)"
                )
                unique_links = list(dict.fromkeys([link for link in links if 'article' in link]))
                print(f"🔗 발견된 링크: {len(unique_links)}개")

                # 3. 개별 기사 수집
                for i, link in enumerate(unique_links):
                    try:
                        await page.goto(link, wait_until="domcontentloaded", timeout=40000)
                        await page.wait_for_timeout(1000)

                        title = await page.locator("h2[class*='ArticleHead_article_title']").first.inner_text()
                        news_date = await page.locator("em.date").first.inner_text()

                        content_loc = page.locator("div._article_content, #comp_news_article")
                        raw_content = await content_loc.first.inner_text()
                        cleaned_content = clean_news_content(raw_content)

                        if len(cleaned_content) > 30:
                            all_news_data.append({
                                "target_date": target_date, # 랭킹 기준일 저장 (추후 분석용)
                                "title": title,
                                "date": news_date,
                                "content": cleaned_content,
                                "url": link
                            })

                        if (i + 1) % 10 == 0:
                            print(f"   ✅ {i+1}번째 기사 완료...")

                    except Exception:
                        continue # 에러 난 기사는 과감히 패스

            except Exception as e:
                print(f"❌ {target_date} 페이지 접속 실패: {e}")

        # 4. 전체 데이터 저장
        if all_news_data:
            df = pd.DataFrame(all_news_data)
            file_name = f"naver_ent_7days_{date_list[0]}_{date_list[-1]}.csv"
            save_path = os.path.join(SAVE_PATH, file_name)
            df.to_csv(save_path, index=False, encoding='utf-8-sig')

            print("\n" + "="*50)
            print(f"🎊 7일치 전체 수집 완료!")
            print(f"📊 총 수집된 기사 수: {len(all_news_data)}개")
            print(f"💾 파일 저장소: {save_path}")
            print("="*50)

        await browser.close()

# 실행
asyncio.run(crawl_7days_news())


📅 [20260122] 날짜 기사 수집 시작...
🔗 발견된 링크: 30개
   ✅ 10번째 기사 완료...
   ✅ 20번째 기사 완료...
   ✅ 30번째 기사 완료...

📅 [20260123] 날짜 기사 수집 시작...
🔗 발견된 링크: 30개
   ✅ 10번째 기사 완료...
   ✅ 20번째 기사 완료...
   ✅ 30번째 기사 완료...

📅 [20260124] 날짜 기사 수집 시작...
🔗 발견된 링크: 30개
   ✅ 10번째 기사 완료...
   ✅ 20번째 기사 완료...
   ✅ 30번째 기사 완료...

📅 [20260125] 날짜 기사 수집 시작...
🔗 발견된 링크: 30개
   ✅ 10번째 기사 완료...
   ✅ 20번째 기사 완료...
   ✅ 30번째 기사 완료...

📅 [20260126] 날짜 기사 수집 시작...
🔗 발견된 링크: 30개
   ✅ 10번째 기사 완료...
   ✅ 20번째 기사 완료...
   ✅ 30번째 기사 완료...

📅 [20260127] 날짜 기사 수집 시작...
🔗 발견된 링크: 30개
   ✅ 10번째 기사 완료...
   ✅ 20번째 기사 완료...
   ✅ 30번째 기사 완료...

📅 [20260128] 날짜 기사 수집 시작...
🔗 발견된 링크: 30개
   ✅ 10번째 기사 완료...
   ✅ 20번째 기사 완료...
   ✅ 30번째 기사 완료...

🎊 7일치 전체 수집 완료!
📊 총 수집된 기사 수: 210개
💾 파일 저장소: /content/drive/MyDrive/DILAB/OK_Data/naver_ent_7days_20260122_20260128.csv
